In [ ]:
!pip uninstall -y unsloth transformers accelerate peft bitsandbytes
!pip install -U pip
!pip install -q "unsloth[colab-new]"


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="train.jsonl",
    split="train"
)


In [ ]:
def format_example(example):
    return {
        "text": f"""<s>[INST] {example['instruction']}

{example['input']} [/INST]
{example['output']}</s>"""
    }
dataset = dataset.map(format_example)
dataset = dataset.remove_columns(
    [col for col in dataset.column_names if col != "text"]
)


In [ ]:
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    max_seq_length=2048,
    dtype=torch.float16,      # IMPORTANT for T4
    load_in_4bit=True,        # QLoRA
)


In [ ]:
def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens
dataset = dataset.map(
    tokenize_function,
    batched=False,
    remove_columns=["text"],
)
print(dataset.column_names)


In [ ]:
from peft import LoraConfig, get_peft_model
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
)


In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./unsloth-mistral-action",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,  # 🔑 FIX
)



In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    tokenizer=tokenizer,
)


In [ ]:
trainer.train()


In [ ]:
model.save_pretrained("meeting-action-lora")
tokenizer.save_pretrained("meeting-action-lora")


In [ ]:
from unsloth import FastLanguageModel
import torch

base_model_name = "mistralai/Mistral-7B-Instruct-v0.2"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_name,
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model.eval()


In [ ]:
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    "meeting-action-lora",  # your saved LoRA directory
)

model.eval()


In [ ]:
FastLanguageModel.for_inference(model)


In [ ]:
prompt = """<s>[INST]
You are a system that extracts action items from meeting transcripts.

Rules:
- Output ONLY valid JSON
- Do NOT add explanations, headings, or bullet points
- Do NOT include text outside JSON
- Use this schema exactly:

{
  "action_items": [
    {
      "action": string,
      "owner": string | null,
      "deadline": string | null
    }
  ]
}

Meeting transcript:
Alex: Please follow up with the client by Friday.
Sam: I will send the email tomorrow.
[/INST]"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **tokenizer(prompt, return_tensors="pt").to("cuda"),
    max_new_tokens=512,
    temperature=0.0,   # 🔑 critical
    do_sample=False,  # 🔑 critical
)



In [ ]:
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Remove everything before the model answer
answer = decoded.split('[/INST]')[-1].strip()

print(answer)


Schema Validity using Pydantic

In [ ]:
eval_ds = load_dataset(
    "json",
    data_files="eval.jsonl",
    split="train"
)

In [ ]:
from pydantic import BaseModel, ValidationError, ConfigDict
from typing import List, Optional

class ActionItem(BaseModel):
    action: str
    owner: Optional[str]
    deadline: Optional[str]

    model_config = ConfigDict(extra="ignore")

class ActionItemsOutput(BaseModel):
    action_items: List[ActionItem]

    model_config = ConfigDict(extra="ignore")


In [ ]:
import json
import re

import json
import ast

def extract_json(text: str):
    if not isinstance(text, str):
        return None

    text = text.strip()

    # 1️⃣ Remove everything before [/INST]
    if "[/INST]" in text:
        text = text.split("[/INST]", 1)[1].strip()

    # 2️⃣ Try JSON
    try:
        return json.loads(text)
    except Exception:
        pass

    # 3️⃣ Try Python literal (THIS is your case)
    try:
        return ast.literal_eval(text)
    except Exception:
        return None


In [ ]:
def clean_action_items(data: dict) -> dict:
    if "action_items" not in data:
        return data

    cleaned = []
    for item in data["action_items"]:
        if isinstance(item, dict):
            item = dict(item)
            item.pop("action_items", None)  # 🔑 REMOVE LEAK
            cleaned.append(item)

    data["action_items"] = cleaned
    return data


In [ ]:
import json
def is_schema_valid(output):
    try:
        print("TYPE OF OUTPUT:", type(output))

        if isinstance(output, dict):
            print("→ Branch: dict")
            data = clean_action_items(output)
            print("CLEANED DATA:", data)
            ActionItemsOutput(**data)
            return True

        if isinstance(output, str):
            print("→ Branch: str")
            data = extract_json(output)
            print("EXTRACTED:", data)
            if data is None:
                return False
            data = clean_action_items(data)
            print("CLEANED DATA:", data)
            ActionItemsOutput(**data)
            return True

        print("→ Branch: neither")
        return False

    except ValidationError as e:
        print(" VALIDATION ERROR:", e)
        return False


In [ ]:
import torch

def run_inference(model, tokenizer, instruction, transcript):
    prompt = f"""<s>[INST] {instruction}

{transcript} [/INST]"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.0,
        do_sample=False,
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    data = extract_json(decoded)

    if data is None:
        print("⚠️ extract_json FAILED")
        return None

    return data


In [ ]:
def schema_validity_rate(model, tokenizer, eval_ds):
    valid = 0

    for ex in eval_ds:
        output = run_inference(
            model,
            tokenizer,
            ex["instruction"],
            ex["input"],
        )

        if is_schema_valid(output):
            valid += 1

    return valid / len(eval_ds)


In [ ]:
def debug_schema_examples(model, tokenizer, eval_ds, n=1):
    for i in range(min(n, len(eval_ds))):
        ex = eval_ds[i]

        print("=" * 80)
        print(f"Example {i+1}")

        print("\n📄 TRANSCRIPT:")
        print(ex["input"][:500], "...\n")

        output = run_inference(
            model,
            tokenizer,
            ex["instruction"],
            ex["input"],
        )

        print("🤖 RAW MODEL OUTPUT:")
        print(output)

        valid = is_schema_valid(output)
        print("\n✅ SCHEMA VALID:", valid)

        # Optional: show parsed JSON if valid
        if valid:
            if isinstance(output, dict):
                parsed = output
            else:
                parsed = extract_json(output)

            print("\n🧩 PARSED JSON:")
            print(parsed)

        print("\n")
debug_schema_examples(model, tokenizer, eval_ds, n=1)


In [ ]:
rate = schema_validity_rate(model, tokenizer, eval_ds)
print(f"Schema validity: {rate:.2%}")


Schema validity: 97.14%

Prompt-only baseline (very important)

In [ ]:
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)


In [ ]:
rate = schema_validity_rate(base_model, base_tokenizer, eval_ds)
print(f"Schema validity: {rate:.2%}")


Schema validity: 0.00%